In [2]:
import pandas as pd
import gurobipy   as      gp
from   gurobipy   import GRB
from MasterProblemOMP import MasterProblem
from SubProblemOMP import SubProblem

AL utilizar ql_t en 0.15 la solución objetivo se convierte en 0.0, estamos usando Ton * %Cu como ql, ¿Esta bien???, como se aplica ql, sobre cada bloque o sobre la suma de estos?, hay que cambiar la función de OMPsubproblem en función de esto 

In [3]:
#path = "C:/Users/willi/OneDrive/Escritorio/Magister/Tesis-Magister/Database/integratedModel/" #Notebook
path = "/home/williams/Tesis-Magister/Databases/"
#path = "C:/Users/Williams Medina/Desktop/Tesis Magister/Tesis-Magister/ThesisCode/MainCode/Databases/integratedModel/" #Desktop
undergroundDatabaseName = 'Modelo_F_OG.xlsx'
#openPitDatabaseName = 'Modelo_F_OG.xlsx'
openPitDatabaseName = 'Modelo_F_OG_4_4_4.xlsx'

In [4]:
if undergroundDatabaseName == openPitDatabaseName:
    undergroundMineDataframe = pd.read_excel(path + undergroundDatabaseName, engine="openpyxl") #Notebook
    openPitDataframe = undergroundMineDataframe
else:
    undergroundMineDataframe = pd.read_excel(path + undergroundDatabaseName, engine="openpyxl") #Notebook
    openPitDataframe = pd.read_excel(path + openPitDatabaseName, engine="openpyxl") #Notebook

In [6]:
class Main:
    def __init__(self, undergroundMineDataframe, openPitDataframe):
        self.openPitDataframe = openPitDataframe
        self.undergroundMineDataframe = undergroundMineDataframe
        self.numberOfPeriods = 20
        self.epsilon = 100000
        
    def execute(self):
        self.createModels()
        self.setMasterProblemCrownPillarHeights()
        self.getResults()
    
    def createModels(self):
        self.createSubProblem()
        self.createMasterProblem()

    def createSubProblem(self):
        self.SubProblem = SubProblem(self.openPitDataframe, self.numberOfPeriods)
        self.SubProblem.setParameters()

    def createMasterProblem(self):
        self.MasterProblem = MasterProblem(self.undergroundMineDataframe, self.numberOfPeriods)
        self.MasterProblem.setParameters()
        
    def setMasterProblemCrownPillarHeights(self):
        self.MasterProblem.V, self.MasterProblem.rho_v = self.SubProblem.V, self.SubProblem.rho_v
        self.MasterProblem.B_v = self.SubProblem.B_v

    def getResults(self):
        it = 0
        self.MasterProblem.setModel()
        currentTheta = -100000000000
        while True:
            it +=1
            self.estimatedW_v, self.estimatedTheta = self.MasterProblem.optimize()
            numberOfInvaiableBlocks = len(self.MasterProblem.B_v[self.MasterProblem.selectedHeight])
            subProblemObjValue, pi_vb = self.SubProblem.execute(numberOfInvaiableBlocks)
            if abs(self.MasterProblem.theta.X - currentTheta) <= self.epsilon:
                subProblemObjValue, pi_vb = self.SubProblem.execute(numberOfInvaiableBlocks, isFinalIteration = False) #esto se cambia si queremos poner toposort
                currentTheta = self.estimatedTheta.X
                self.estimatedW_v, self.estimatedTheta = self.MasterProblem.optimize()
                print()
                print("-----------------------------Valor óptimo encontrado-----------------------------")
                print("Optimal Crown Pillar height: {}".format(self.estimatedW_v))
                print("Optimal ObjVal: {}".format(self.MasterProblem.undergroundModel.objVal))
                objVal = self.MasterProblem.undergroundModel.objVal
                toposort = self.MasterProblem.undergroundModel.objVal- self.MasterProblem.theta.X + subProblemObjValue
                print("Optimal Theta: {}".format(self.MasterProblem.theta.X))
                print("Optimal ObjVal TopoSort: {}".format(self.MasterProblem.undergroundModel.objVal- self.MasterProblem.theta.X + subProblemObjValue))
                print(f"GAP: {100 * (objVal-toposort)/objVal}")
                #print("Current theta: {}".format(currentTheta))
                break
            
            print(it, "Objective Value =", self.MasterProblem.undergroundModel.objVal)
            print("w_v: {}".format(self.estimatedW_v))
            print("Master theta: {}".format(self.MasterProblem.theta))
            print("Current theta: {}".format(currentTheta))
            currentTheta = self.estimatedTheta.X
            self.MasterProblem.undergroundModel.addConstr(self.MasterProblem.theta <= subProblemObjValue + gp.quicksum(gp.quicksum(pi_vb[t] * len(self.MasterProblem.B_v[v]) * (self.estimatedW_v[v] - self.MasterProblem.w_v[v]) for t in self.MasterProblem.t_S) for v in self.MasterProblem.V))
            print()
            print()

In [7]:
main = Main(undergroundMineDataframe, openPitDataframe)
main.execute()

Set parameter Username
Academic license - for non-commercial use only - expires 2023-10-02
Set parameter TimeLimit to value 3600
1 Objective Value = 2025218384.819684
w_v: {745: 1.0, 785: 0.0, 825: 0.0, 865: 0.0, 905: 0.0, 945: 0.0, 985: 0.0, 1025: 0.0, 1065: 0.0, 1105: 0.0, 1145: 0.0, 1185: 0.0, 1225: 0.0, 1265: 0.0, 1305: 0.0}
Master theta: <gurobi.Var theta (value 800000000.0)>
Current theta: -100000000000


2 Objective Value = 1624948384.8196843
w_v: {745: 1.0, 785: 0.0, 825: 0.0, 865: 0.0, 905: 0.0, 945: 0.0, 985: 0.0, 1025: 0.0, 1065: 0.0, 1105: 0.0, 1145: 0.0, 1185: 0.0, 1225: 0.0, 1265: 0.0, 1305: 0.0}
Master theta: <gurobi.Var theta (value 399730000.0)>
Current theta: 800000000.0



-----------------------------Valor óptimo encontrado-----------------------------
Optimal Crown Pillar height: {745: 1.0, 785: 0.0, 825: 0.0, 865: 0.0, 905: 0.0, 945: 0.0, 985: 0.0, 1025: 0.0, 1065: 0.0, 1105: 0.0, 1145: 0.0, 1185: 0.0, 1225: 0.0, 1265: 0.0, 1305: 0.0}
Optimal ObjVal: 1624948384.81

10 desagregado

Optimal ObjVal: 2343184937.4682417
Optimal Theta: 640110000.0
Optimal ObjVal TopoSort: 2329594937.4682417
GAP: 0.5799798292781656

5 desagregado

Optimal ObjVal: 2840469882.9332986
Optimal Theta: 781890000.0
Optimal ObjVal TopoSort: 2830099882.9332986
GAP: 0.3650804418771411

15 agregado

Optimal Crown Pillar height: {745: 1.0, 785: 0.0, 825: 0.0, 865: 0.0, 905: 0.0, 945: 0.0, 985: 0.0, 1025: 0.0, 1065: 0.0, 1105: 0.0, 1145: 0.0, 1185: 0.0, 1225: 0.0, 1265: 0.0, 1305: 0.0}
Optimal ObjVal: 1903625419.951932
Optimal Theta: 469290000.0
Optimal ObjVal TopoSort: 1884355419.951932
GAP: 1.0122789808347161

In [8]:
openPitDataframe = undergroundMineDataframe
main = Main(undergroundMineDataframe, openPitDataframe)
main.execute()

Set parameter TimeLimit to value 3600
1 Objective Value = 2025218384.819684
w_v: {530: 1.0, 540: 0.0, 550: 0.0, 560: 0.0, 570: 0.0, 580: 0.0, 590: 0.0, 600: 0.0, 610: 0.0, 620: 0.0, 630: 0.0, 640: 0.0, 650: 0.0, 660: 0.0, 670: 0.0, 680: 0.0, 690: 0.0, 700: 0.0, 710: 0.0, 720: 0.0, 730: 0.0, 740: 0.0, 750: 0.0, 760: 0.0, 770: 0.0, 780: 0.0, 790: 0.0, 800: 0.0, 810: 0.0, 820: 0.0, 830: 0.0, 840: 0.0, 850: 0.0, 860: 0.0, 870: 0.0, 880: 0.0, 890: 0.0, 900: 0.0, 910: 0.0, 920: 0.0, 930: 0.0, 940: 0.0, 950: 0.0, 960: 0.0, 970: 0.0, 980: 0.0, 990: 0.0, 1000: 0.0, 1010: 0.0, 1020: 0.0, 1030: 0.0, 1040: 0.0, 1050: 0.0, 1060: 0.0, 1070: 0.0, 1080: 0.0, 1090: 0.0, 1100: 0.0, 1110: 0.0, 1120: 0.0, 1130: 0.0, 1140: 0.0, 1150: 0.0, 1160: 0.0, 1170: 0.0, 1180: 0.0, 1190: 0.0, 1200: 0.0, 1210: 0.0, 1220: 0.0, 1230: 0.0, 1240: 0.0, 1250: 0.0, 1260: 0.0, 1270: 0.0, 1280: 0.0, 1290: 0.0, 1300: 0.0, 1310: 0.0, 1320: 0.0}
Master theta: <gurobi.Var theta (value 800000000.0)>
Current theta: -100000000000


2

In [9]:
main.MasterProblem.undergroundModel.getAttr(GRB.Attr.X, main.MasterProblem.undergroundModel.getVars())

[1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0

In [10]:
len(main.MasterProblem.undergroundBlocks)


1003520